In [1]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import torch

# 1. Konfiguracja
COLLECTION_NAME = "labor_code_pl"
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 2. Inicjalizacja klienta Qdrant i modelu
client = QdrantClient(host="localhost", port=6333)
model = SentenceTransformer(model_name, device=device)

print(f"Połączono z Qdrant. Model załadowany na: {device.upper()}")

C:\Users\Greg Z\AppData\Local\Temp\ipykernel_12076\2697010447.py:11: UserWarning: Qdrant client version 1.13.0 is incompatible with server version 1.16.3. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  client = QdrantClient(host="localhost", port=6333)


Połączono z Qdrant. Model załadowany na: CUDA


In [2]:
def search_labor_code(query, limit=3):
    # 1. Zamiana pytania na wektory (Embeddingi)
    query_vector = model.encode(query).tolist()

    # 2. Wyszukiwanie w Qdrant
    search_result = client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=limit,
        with_payload=True ## żeby widzieć treść artykułów
    ).points

    # 3. Wyświetlenie wyników
    print(f"Wyniki dla zapytania: '{query}'\n")
    print("="*40)
    
    for i, res in enumerate(search_result):
        score = res.score
        payload = res.payload
        print(f"TOP {i+1} | {payload.get('art_id')} | Podobieństwo: {score:.4f}")
        print(f"Treść: {payload.get('content')[:300]}...") ### pierwsze 300 znaków
        print("-" * 40)

print("Funkcja search_labor_code jest gotowa.")

Funkcja search_labor_code jest gotowa.


# Testy

In [4]:
# Test 1: Pytanie wprost
search_labor_code("Ile dni urlopu wypoczynkowego przysługuje pracownikowi?", limit=2)

# Test 2: Sytuacja życiowa (brak słowa 'urlop' w pytaniu - test semantyki)
search_labor_code("Chcę wziąć wolne na ślub, jakie mam prawa?", limit=2)

# Test 3: Pytanie o obowiązki (nadgodziny)
search_labor_code("Czy muszę pracować dłużej niż 8 godzin dziennie?", limit=2)

Wyniki dla zapytania: 'Ile dni urlopu wypoczynkowego przysługuje pracownikowi?'

TOP 1 | Art. 154 | Podobieństwo: 0.8185
Treść: Art. 154. § 1. Wymiar urlopu wynosi: 
1) 20 dni – jeżeli pracownik jest zatrudniony krócej niż 10 lat; 
2) 26 dni – jeżeli pracownik jest zatrudniony co najmniej 10 lat. 
 
 
§ 2. Wymiar urlopu dla pracownika zatrudnionego w  niepełnym wymiarze 
czasu pracy ustala się proporcjonalnie do wymiaru czas...
----------------------------------------
TOP 2 | Art. 162 | Podobieństwo: 0.8036
Treść: Art. 162. Na wniosek pracownika urlop może być podzielony na części. 
W takim jednak przypadku co najmniej jedna część wypoczynku powinna trwać 
nie mniej niż 14 kolejnych dni kalendarzowych....
----------------------------------------
Wyniki dla zapytania: 'Chcę wziąć wolne na ślub, jakie mam prawa?'

TOP 1 | Art. 177 | Podobieństwo: 0.3979
Treść: Art. 177. § 1. W okresie ci ąży oraz w  okresie urlopu macierzy ńskiego, 
a także od dnia z łożenia przez pracownic ę lub pracown